# Agregate, Summarize and Group data

> 1. Simple Agregations
> - Group by 
> - Agg ()


In [1]:
#Import libraries
from pandasql import sqldf #!pip install -U pandasql
pysqldf = lambda q: sqldf(q, globals())
from collections import Counter
import pandas as pd #Pandas Lib
import numpy as np  #NumPy Lib
import matplotlib.pyplot as plt
import urllib.request, json 
from IPython.display import display
from IPython.display import Image

In [9]:
url = "https://data.medicare.gov/api/views/9n3s-kdb3/rows.csv?accessType=DOWNLOAD" 
life = pd.read_csv(url)

Let's read in our own data set from the plethora of data available on the Govt's website:
*data owned and sourced from: https://www.data.gov/*

In [11]:
life.head(1) 

,Hospital Name,Provider Number,State,Measure Name,Number of Discharges,Footnote,Excess Readmission Ratio,Predicted Readmission Rate,Expected Readmission Rate,Number of Readmissions,Start Date,End Date
0,HIGHLANDS MEDICAL CENTER,10061,AL,READM-30-AMI-HRRP,Not Available,1.0,Not Available,Not Available,Not Available,Not Available,01-JUL-13,30-JUN-16


### 1. Simple Agregations

We have already covered many of the topics in lessons 1 and 2. I have not given the SAS and SQL code in all examples becuase this is intuitive and much of it is basic SQL or PROC SUMMARY.

**How many rows in the dataset**

~~~
SAS: you should be able to see the number of obs in the log.
SQL: select count (*) from table;
~~~

In [12]:
life['State'].count()

19830

**Largest Value in the column?**
~~~
select max('Number of Discharges') from table;
~~~

In [69]:
df=life
cols_to_convert = ['Number of Discharges', 'Expected Readmission Rate', 'Predicted Readmission Rate']
cols_to_date    = ['Start Date', 'End Date']
for col in cols_to_convert:
    df[col] = pd.to_numeric(df[col], errors='coerce')
for col in cols_to_date:
    df[col] = pd.to_datetime(df[col], errors='coerce')      

df.dtypes

df['Number of Discharges'].max()

8355.0

**How many discharges of phone calls are in the State of AL**
~~~
select sum ('Number of Discharges') 
from table 
where 'State' = 'AL' 
~~~

In [20]:
df['Number of Discharges'][df[ 'State'] == 'AL'].sum()

95303.0

In [21]:
#what if we wanted highland medical center?
df['Number of Discharges'][df[ 'Hospital Name'] == 'HIGHLANDS MEDICAL CENTER'].sum()

820.0

**How many entries are there for each state?**

~~~
SQL
select 'State', 
        count (*) 
from table 
group by 'State'

SAS:
proc freq data = table;
tables 'State';
run;
~~~

In [26]:
x = df['State'].value_counts()
x.head()

TX    1872
CA    1782
FL    1026
NY     912
PA     900
Name: State, dtype: int64

**Number of non-null unique network entries for hospitals**
~~~
select count (distinct 'Hospital Name') from table;
~~~

In [24]:
df['Hospital Name'].nunique()  

3195

In [25]:
#How many rows the dataset
df['Hospital Name'].count()

19830

### 2. Group By

In SAS we use PROC SUMMARY or MEANS with BY, VAR, and CLASS statements to summarize data. While SQL uses group by statements and each variable is summarized in the select statement. We will go through simple group by statements and the explore the *agg()* function to sumarize different columns to create data sets. 

**Assign values to a dictionairy **

In [28]:
df.groupby(['State']).groups.keys() 

dict_keys(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'])

In [33]:
len (df.groupby(['State']).groups['CA'])

1782

**First Record by State**

In [79]:
df.groupby('State').first().head(5)

,Hospital Name,Provider Number,Measure Name,Number of Discharges,Footnote,Excess Readmission Ratio,Predicted Readmission Rate,Expected Readmission Rate,Number of Readmissions,Start Date,End Date
State,,,,,,,,,,,
AK,PROVIDENCE ALASKA MEDICAL CENTER,20001,READM-30-CABG-HRRP,118.0,5.0,0.8939,12.1,13.5,11,2013-07-01,2016-06-30
AL,HIGHLANDS MEDICAL CENTER,10061,READM-30-AMI-HRRP,103.0,1.0,Not Available,14.4,14.6,Not Available,2013-07-01,2016-06-30
AR,NORTH METRO MEDICAL CENTER,40074,READM-30-HIP-KNEE-HRRP,637.0,5.0,Not Available,14.2,13.9,Not Available,2013-07-01,2016-06-30
AZ,ABRAZO CENTRAL CAMPUS,30030,READM-30-CABG-HRRP,119.0,1.0,0.9867,13.4,13.6,Too Few to Report,2013-07-01,2016-06-30
CA,MILLS-PENINSULA MEDICAL CENTER,50007,READM-30-PN-HRRP,525.0,5.0,0.8116,12.9,15.9,56,2013-07-01,2016-06-30


**Sum of Discharges by State**

In [37]:
x = df.groupby('State')['Number of Discharges'].sum()
x.head()

State
AK      5019.0
AL     95303.0
AR     61703.0
AZ     76353.0
CA    303151.0
Name: Number of Discharges, dtype: float64

**Get the number of Hospitals in Each State**

In [57]:
df.groupby('State')['Hospital Name'].count().head()

State
AK      48
AL     510
AR     270
AZ     378
CA    1782
Name: Hospital Name, dtype: int64

**For the State of AL, list the number of discharges by hospital name**

In [43]:
x= df[df['State'] == 'AL'].groupby('Hospital Name')['Number of Discharges'].sum()
x.head()

Hospital Name
ANDALUSIA REGIONAL HOSPITAL      690.0
ATHENS LIMESTONE HOSPITAL        865.0
ATMORE COMMUNITY HOSPITAL        442.0
BAPTIST MEDICAL CENTER EAST      910.0
BAPTIST MEDICAL CENTER SOUTH    2238.0
Name: Number of Discharges, dtype: float64

**By STATE and Measure Name count the number of discharges**

In [47]:
x=df.groupby(['State', 'Measure Name'])['Number of Discharges'].count()
x.head()

State  Measure Name          
AK     READM-30-AMI-HRRP         2
       READM-30-CABG-HRRP        1
       READM-30-COPD-HRRP        6
       READM-30-HF-HRRP          5
       READM-30-HIP-KNEE-HRRP    3
Name: Number of Discharges, dtype: int64

In [52]:
x= df.groupby(['State', 'Hospital Name'])['Number of Discharges'].sum()
x.head()

State  Hospital Name                     
AK     ALASKA NATIVE MEDICAL CENTER           425.0
       ALASKA REGIONAL HOSPITAL              1046.0
       BARTLETT REGIONAL HOSPITAL               NaN
       CENTRAL PENINSULA GENERAL HOSPITAL     565.0
       FAIRBANKS MEMORIAL HOSPITAL            219.0
Name: Number of Discharges, dtype: float64

**Sum of all discharges by state**

In [64]:
df.groupby('State').agg({'Number of Discharges': ['sum']}).head()

,Number of Discharges
,sum
State,
AK,5019.0
AL,95303.0
AR,61703.0
AZ,76353.0
CA,303151.0


**Sum & Mean of all Discharges & Readmission Rate | BY | State and Meassure Name**

In [65]:
df.groupby(['State', 'Measure Name']).agg({'Number of Discharges': ['sum', 'mean'],      # find the sum of the discharges for each group
                                           'Predicted Readmission Rate': ['mean'] # find the number of network type entries
                                             }).head(10)  


Number of Discharges              \
                                              sum        mean   
State Measure Name                                              
AK    READM-30-AMI-HRRP                     556.0  278.000000   
      READM-30-CABG-HRRP                    118.0  118.000000   
      READM-30-COPD-HRRP                    834.0  139.000000   
      READM-30-HF-HRRP                      830.0  166.000000   
      READM-30-HIP-KNEE-HRRP               1330.0  443.333333   
      READM-30-PN-HRRP                     1351.0  225.166667   
AL    READM-30-AMI-HRRP                    8394.0  262.312500   
      READM-30-CABG-HRRP                   3328.0  175.157895   
      READM-30-COPD-HRRP                  18904.0  286.424242   
      READM-30-HF-HRRP                    22146.0  369.100000   

                             Predicted Readmission Rate  
                                                   mean  
State Measure Name                                       
AK    READM-30-AMI-HRRP                       13.580000  
      READM-30-CABG-HRRP                      12.700000  
      READM-30-COPD-HRRP                      17.212500  
      READM-30-HF-HRRP                        19.637500  
      READM-30-HIP-KNEE-HRRP                   3.742857  
      READM-30-PN-HRRP                        14.487500  
AL    READM-30-AMI-HRRP                       15.850000  
      READM-30-CABG-HRRP                      13.631818  
      READM-30-COPD-HRRP                      17.997436  
      READM-30-HF-HRRP                        20.532468

In [68]:
# Define the aggregation procedure outside of the groupby operation
AGY = {
    'Number of Discharges':'sum',
    'Predicted Readmission Rate': lambda x: max(x) - 1
    }

df.groupby('State').agg(AGY).head()

,Number of Discharges,Predicted Readmission Rate
State,,
AK,5019.0,21.5
AL,95303.0,NaN
AR,61703.0,NaN
AZ,76353.0,24.5
CA,303151.0,28.0


In [73]:
# find the min, max, and sum of the Discharge column
df.groupby(['State']).agg({'Number of Discharges': [min, max, sum, 'mean'],      
                              'Start Date': [min, 'first', 'nunique']}).head()    
# get the min, first, and number of unique dates per state

Number of Discharges                               Start Date  \
                       min     max       sum        mean        min   
State                                                                 
AK                    76.0   608.0    5019.0  218.217391 2013-07-01   
AL                     0.0  1861.0   95303.0  350.378676 2013-07-01   
AR                    48.0  2152.0   61703.0  358.738372 2013-07-01   
AZ                     0.0  2165.0   76353.0  353.486111 2013-07-01   
CA                     0.0  3073.0  303151.0  314.471992 2013-07-01   

                          
           first nunique  
State                     
AK    2013-07-01       1  
AL    2013-07-01       1  
AR    2013-07-01       1  
AZ    2013-07-01       1  
CA    2013-07-01       1